In [3]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# Load your dataset
# Make sure to replace 'your_dataset.csv' with your actual dataset file
data = pd.read_csv('heart_disease.csv')

# Preprocessing
data.fillna(data.mean(), inplace=True)  # Simple imputation for missing values

# Define features and target variable
X = data.drop('target', axis=1)  # Replace 'target' with your target column name
y = data['target']  # Binary target variable (0 for no heart disease, 1 for heart disease)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [7]:

# Build the feedforward neural network model
# Build the feedforward neural network model
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer as a tuple
    layers.Dense(64, activation='relu'),    # Hidden layer 1
    layers.Dense(32, activation='relu'),    # Hidden layer 2
    layers.Dense(1, activation='sigmoid')    # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

# Risk level classification function
def classify_risk(prob):
    """Classify the risk level based on the predicted probability."""
    if prob > 0.7:
        return 'High Risk'
    elif prob > 0.4:
        return 'Medium Risk'
    else:
        return 'Low Risk'

# Define the prediction function with risk classification
def predict_heart_disease(age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal):
    input_data = np.array([age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal]).reshape(1, -1)
    input_data = scaler.transform(input_data)  # Standardize the input data
    prob = model.predict(input_data)[0][0]  # Get the probability of heart disease
    risk_level = classify_risk(prob)  # Classify the risk level based on the predicted probability

    if prob < 0.5:
        result = "No heart disease present"
    else:
        result = "Heart disease is present"

    # Include the risk percentage in the output
    risk_percentage = round(prob * 100, 2)  # Convert to percentage
    return f"{result}. Risk Level: {risk_level} ({risk_percentage}%)"


Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.4188 - loss: 0.7363 - val_accuracy: 0.6279 - val_loss: 0.6818
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6769 - loss: 0.6013 - val_accuracy: 0.6744 - val_loss: 0.5792
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8026 - loss: 0.4920 - val_accuracy: 0.6744 - val_loss: 0.5215
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8250 - loss: 0.4152 - val_accuracy: 0.6744 - val_loss: 0.4924
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8150 - loss: 0.3695 - val_accuracy: 0.7209 - val_loss: 0.4785
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8703 - loss: 0.3469 - val_accuracy: 0.7209 - val_loss: 0.4771
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8628 - loss: 0.3249 - val_accuracy: 0.7209 - val_loss: 0.4875
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8632 - loss: 0.2813 - val_accuracy: 0.7442 - 

In [9]:

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_heart_disease,
    inputs=[
        gr.Number(label="Age"),
        gr.Radio([0, 1], label="Sex (0 = Female, 1 = Male)"),
        gr.Number(label="Chest Pain Type (0-3)"),
        gr.Number(label="Resting Blood Pressure"),
        gr.Number(label="Serum Cholesterol (mg/dl)"),
        gr.Radio([0, 1], label="Fasting Blood Sugar > 120 mg/dl (1 = True, 0 = False)"),
        gr.Number(label="Resting ECG results (0-2)"),
        gr.Number(label="Max Heart Rate Achieved"),
        gr.Radio([0, 1], label="Exercise Induced Angina (1 = Yes, 0 = No)"),
        gr.Number(label="ST Depression Induced by Exercise"),
        gr.Number(label="Slope of the Peak Exercise ST Segment (0-2)"),
        gr.Number(label="Number of Major Vessels (0-3) Colored by Fluoroscopy"),
        gr.Number(label="Thalassemia (1 = Normal; 2 = Fixed Defect; 3 = Reversible Defect)")
    ],
    outputs="text",
    title="Heart Disease Prediction",
    description="Enter the patient's data to predict whether they have heart disease, including risk level."
)

# Launch the interface
interface.launch()


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
